In [1]:
import openai
import json
import os
import pprint

In [2]:
client = openai.OpenAI()

In [20]:
system_content = """
You are a data researcher and extractor, capable of browsing the web for data and delivering it in a semi-structured manner.
"""

user_content = """
I'd like your help with data research and extraction. I want you to get data about the Oscar winners for Best Actress from 2020 to 2024.

For each winner, I'd like to know the following info:
- year: Year of the awards
- ceremony_number: Number of the ceremony
- full_name: Full name
- artistic_name: Artistic name
- country_birth: Country she was born in
- countries_citizenship: Country(s) she has citizenship to
- role: Name of the role(s) she played
- movie: Name of the movie(s) she was awarded for
- release_year: Year in which the movie was released
- main_production_country: Main country where said movie(s) were produced
- all_production_countries: All countries involved in the production of the movie
- main_language: Main language spoken in said movie(s)
- all_languages: All languages credited as spoken in the movie(s)
- source_ceremony: Source for the info on the ceremony (e.g. "https://en.wikipedia.org/wiki/94th_Academy_Awards")
- source_actress: Source for the info on the actress (e.g. "https://en.wikipedia.org/wiki/Frances_McDormand")
- source_movie: Source for the info on the movie (e.g. "https://en.wikipedia.org/wiki/Nomadland")

Also, please follow these instructions:
- Prioritize Wikipedia as the source from this data;
- Return the data as a list of dictionaries that can be formatted as JSON;
- Return ONLY the relevant and formatted data, without any text before or after it, not even "```json" or similar.
"""

In [21]:
completion = client.chat.completions.create(
    model="gpt-4o-mini",
    messages=[
        {"role": "system", "content": system_content},
        {"role": "user", "content": user_content}
    ]
)

In [22]:
print(completion.choices[0].message.content)

[
    {
        "year": 2020,
        "ceremony_number": 92,
        "full_name": "Renée Kathleen Zellweger",
        "artistic_name": "Renée Zellweger",
        "country_birth": "United States",
        "countries_citizenship": "United States",
        "role": "Judy Garland",
        "movie": "Judy",
        "release_year": 2019,
        "main_production_country": "United Kingdom",
        "all_production_countries": "United Kingdom, United States",
        "main_language": "English",
        "all_languages": "English",
        "source_ceremony": "https://en.wikipedia.org/wiki/92nd_Academy_Awards",
        "source_actress": "https://en.wikipedia.org/wiki/Ren%C3%A9e_Zellweger",
        "source_movie": "https://en.wikipedia.org/wiki/Judy_(film)"
    },
    {
        "year": 2021,
        "ceremony_number": 93,
        "full_name": "Frances McDormand",
        "artistic_name": "Frances McDormand",
        "country_birth": "United States",
        "countries_citizenship": "United States",

In [23]:
type(completion.choices[0].message.content)

str

In [24]:
lililist = json.loads(completion.choices[0].message.content)

In [28]:
with open('best_actress_oscar_winners/raw_2020-2024.json', 'w', encoding='UTF-8') as f:
    json.dump(lililist, f)

In [29]:
with open('best_actress_oscar_winners/raw_2020-2024.json', 'r') as f:
    fffffile = json.load(f)

In [30]:
fffffile

[{'year': 2020,
  'ceremony_number': 92,
  'full_name': 'Renée Kathleen Zellweger',
  'artistic_name': 'Renée Zellweger',
  'country_birth': 'United States',
  'countries_citizenship': 'United States',
  'role': 'Judy Garland',
  'movie': 'Judy',
  'release_year': 2019,
  'main_production_country': 'United Kingdom',
  'all_production_countries': 'United Kingdom, United States',
  'main_language': 'English',
  'all_languages': 'English',
  'source_ceremony': 'https://en.wikipedia.org/wiki/92nd_Academy_Awards',
  'source_actress': 'https://en.wikipedia.org/wiki/Ren%C3%A9e_Zellweger',
  'source_movie': 'https://en.wikipedia.org/wiki/Judy_(film)'},
 {'year': 2021,
  'ceremony_number': 93,
  'full_name': 'Frances McDormand',
  'artistic_name': 'Frances McDormand',
  'country_birth': 'United States',
  'countries_citizenship': 'United States',
  'role': 'Fern',
  'movie': 'Nomadland',
  'release_year': 2020,
  'main_production_country': 'United States',
  'all_production_countries': 'United 